In [25]:
import yfinance as yf
df = yf.download("INVE-B.ST", start="2015-01-01", progress=False)
df.reset_index().to_csv("INVE-B_clean.csv", index=False)
print("Data klar!")

C:\Users\Harout\AppData\Local\Temp\ipykernel_10108\2978242015.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("INVE-B.ST", start="2015-01-01", progress=False)


Data klar!


In [26]:
import backtrader as bt
import pandas as pd

class GoldenCross(bt.Strategy):
    params = (('short', 20), ('long', 50),)

    def _init_(self):
        self.sma_short = bt.ind.SMA(period=self.p.short)
        self.sma_long  = bt.ind.SMA(period=self.p.long)
        self.crossover = bt.ind.CrossOver(self.sma_short, self.sma_long)

    def next(self):
        if not self.position and self.crossover > 0:
            self.buy(size=self.broker.getcash() // self.data.close[0])
        elif self.position and self.crossover < 0:
            self.close()

data = bt.feeds.GenericCSVData(
    dataname="INVE-B_clean.csv",
    dtformat=('%Y-%m-%d'),
    datetime=0, open=1, high=2, low=3, close=4, volume=5, openinterest=-1
)

cerebro = bt.Cerebro()
cerebro.addstrategy(GoldenCross)
cerebro.adddata(data)
cerebro.broker.setcash(100000)
cerebro.broker.setcommission(commission=0.0005)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')

print("Kör backtest...")
results = cerebro.run()[0]

final = cerebro.broker.getvalue()
sharpe = results.analyzers.sharpe.get_analysis().get('sharperatio', 0)
maxdd = results.analyzers.drawdown.get_analysis()['max']['drawdown']
total_trades = results.analyzers.trades.get_analysis()['total']['closed']

print(f"\nKLART! INVE-B.ST 2015→2025")
print(f"100 000 kr → {final:,.0f} kr")
print(f"Total avkastning: +{(final/100000-1)*100:.1f}%")
print(f"Sharpe-ratio: {sharpe:.2f}")
print(f"Max drawdown: {maxdd:.1f}%")
print(f"Antal trades: {total_trades}")

Kör backtest...


ValueError: time data '' does not match format '%Y-%m-%d'